<a href="https://colab.research.google.com/github/modichirag/flowpm/blob/lensing/notebooks/Denise_lightcone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pylab inline 
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline as iuspline
from flowpm.tfpower import linear_matter_power
from flowpm.tfbackground import cosmo,z2a,a2z,afactor, chifactor,rad_comoving_distance
from flowpm.raytracing import  lightcone, Born, wlen, cons, A, nbar_
import flowpm.constants as constants
from flowpm.angular_power import radial_profile, measure_power_spectrum, pixel_size
from flowpm.angular_power_tf import measure_power_spectrum_tf
#from flowpm.spectrum_for_cosmology import power_spectrum_for_cosmology
import flowpm  
import time
start_time = time.time()

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# You may need to adapt this path depending on where you are running the notebook
#This is the power spectrum of initial conditions
klin = np.loadtxt('/Users/dl264294/Desktop/github/flowpm/flowpm/data/Planck15_a1p00.txt').T[0]
plin=linear_matter_power(cosmo, klin)
ipklin = iuspline(klin, plin)

In [2]:
nc=[64,64,640]   # size of the cube, number of cells
nc_xy=64                    # number of pixel for x and  y 
Boxsize=[200,200,2000]          # Physical size of the cube

In [3]:
#To make lens planes of size 200 Mpc/h :
r = np.linspace(0,2000,10, endpoint=True)
a = afactor(r)  

In [ ]:
# This allows us to go to roughly z=1
plot(r,a2z(a), '+')
ylabel(r'z')
xlabel(r'Mpc/h')

In [ ]:
# We will first run the simulation to the lowest scaler factor entering the lightcone
init_stages = np.linspace(0.1, a[-1], 4, endpoint=True)
initial_conditions = flowpm.linear_field(nc,    
                                        Boxsize, 
                                         ipklin,         
                                         batch_size=1)

# Sample particles
state = flowpm.lpt_init(initial_conditions, 0.1)   
# Evolve particles down to z=0
med_state = flowpm.nbody(state, init_stages, nc)         
# Retrieve final density field
med_field = flowpm.cic_paint(tf.zeros_like(initial_conditions), med_state[0])

In [ ]:
# At this stage we are at the right edge of the lightcone
figure(figsize=[20,5])
imshow(tf.reshape(med_field, nc).numpy().sum(axis=0))

In [ ]:
final_state, lps_a, lps = lightcone(med_state, a[::-1], 
                                  nc, 
                                    5.*60/nc_xy, nc_xy,cosmo)




In [ ]:
#Let's define the source's redshift
#zs=0.7755102040816326
a_s=z2a(1.00)
ds=chifactor(a_s)

In [ ]:
#Retrieve final density field
final_field = flowpm.cic_paint(tf.zeros_like(initial_conditions), final_state[0])

# And this is what remains of the simulation at redhift=0 
figure(figsize=[20,5])
imshow(tf.reshape(final_field, [nc_xy,nc_xy,-1]).numpy().sum(axis=0)) 

In [ ]:
# Here are the lens planes exported during the simulation
figure(figsize=(20,5))
for i in range(len(lps_a)):
    subplot(1,9,i+1)
    imshow(lps[i][0]);
    title('z = %0.2f'%a2z(lps_a[i]))


In [ ]:
final_state.shape   # tensor of shape (3, batch_size, npart, 3)

In [ ]:
k_map=Born(lps_a,lps,ds,nc,Boxsize,nc_xy,5.,cosmo)

In [ ]:
# from astropy.io import fits
# hdu = fits.PrimaryHDU(k_map)
# hdul = fits.HDUList([hdu])
# hdul.writeto(('.fits')

In [ ]:
time.time() - start_time

In [ ]:
imshow(k_map)
colorbar()
#savefig('kmap_denise_z1.png',dpi=100)


In [ ]:
#begin the computation of power spectrum from the map

In [ ]:
#ell = 2. * np.pi * k / pixel_size / 512
ell1, ps_example1 = measure_power_spectrum(k_map,5.,64)
ell2, ps_example2 = measure_power_spectrum_tf(tf.cast(k_map,dtype=tf.complex64),5.,64)


In [ ]:
loglog(ell1, ps_example1, label='kappaTNG map')
loglog(ell2, ps_example2, label='kappaTNG map')
xlabel('$\ell$')
ylabel('Ps')
legend()

In [ ]:
#begin computation power spectra from theory 

In [ ]:
import jax
import jax_cosmo as jc

In [ ]:
z = linspace(0,2,100)
pz = zeros_like(z)
pz[50] =1. 
nzs_s=jc.redshift.kde_nz(z, pz, bw=0.05)
# let's draw the nz on a new array of redshifts
zsamp = np.linspace(0,2,128)
plot(zsamp, nzs_s(zsamp))

In [ ]:
nzs = [nzs_s]

In [ ]:
probes = [ jc.probes.WeakLensing(nzs, sigma_e=0.26) ]

In [ ]:
elle = np.logspace(1,4) # Defines a range of \ell
cosmo_jc = jc.Planck15()
# And compute the data vector
cls = jc.angular_cl.angular_cl(cosmo_jc, elle, probes)

In [ ]:

%pylab inline 
loglog(elle, cls[0])
ylabel(r'$C_\ell$')
xlabel(r'$\ell$');
title(r'Angular $C_\ell$');

In [ ]:
loglog(elle, cls[0],label='T. Power Spectrum')
#loglog(ell1, ps_example1, label='kappa map')
ylabel(r'$C_\ell$')
xlabel(r'$\ell$')
#xlim(10,10000)
legend()
title('nc=128c')
#savefig('power_con128_z=0.99.png',dpi=80)

In [ ]:
print("--- %d seconds ---(%d minutes)" % ((time.time() - start_time),(time.time() - start_time)/60.))

In [ ]:
%pylab inline 
loglog(ell, power_spectrum)
#loglog(elle, cls[0],label='T. Power Spectrum')
#loglog(ell1, ps_example1, label='kappa map')
ylabel(r'$C_\ell$')
xlabel(r'$\ell$');
title(r'Angular $C_\ell$')

In [ ]:
loglog(ell1, ps_example1, label='kappaTNG map')
loglog(ell2, ps_example2, label='kappaTNG map')
loglog(ell3, power_spectrum3, label='kappaTNG map')
xlabel('$\ell$')
ylabel('Ps')
legend()